# AlphaQUBO Tutoral

This tutorial shows how to read an AlphaQUBO formatted file, process it as a D-Wave BinaryQuadraticModel, and submit to an AlphaQUBO solver.

The AlphaQUBO file format is a text file with information on the first line that describes the size of the q(i,j) matrix.

1. Program line is marked by a “**p**” in the first column. A single program line must be the first line in the file. The program line has two arguments: **variable_count** and **non_zero_element_count**

```
    p 50 225
```

2. The remaining lines are made up of three numbers, separated by one or more blanks. The first two numbers, ($i$ and $j$), are the indices for this Q matrix entry, where ($i <= j$). Each index must be in the range {1, **variable_count**}. The third number is the value of the Q matrix at $Q(i,j)$, specified as an integer or floating point value

#### Example
```
 p  100  475
 1  35  -19
 1  44  -22
 1  47  27
 1  49  -66
 1  58  -69
 1  64  63
 1  72  -89
 1  73  -19
 1  74  -69
 1  76  -12
 1  84  40
 1  98  33
 2  2  52
```


In [1]:
import sys
import alphaqubo_client as aq
from alphaqubo_client.rest import ApiException
import dimod
import neal
from dwave_qbsolv import QBSolv

## Read AlphaQUBO formatted file as BinaryQuadraticModel

In [2]:
def read_alphaqubo_bqm(filename:str):
    lines = []
    with open(filename, "rt") as myfile:
        for line in myfile:
            lines.append(line.rstrip('\n'))
    Q = {}
    for line in lines:
        p = line.split()
        if len(p) > 0:
            if p[0] == 'c':
                continue
            elif p[0] == 'p':
                nVars = int(p[1])
            else:
                if len(p) == 3:
                    i = int(p[0]) - 1
                    j = int(p[1]) - 1
                    w = float(p[2])
                    if i != j:
                        w *= 2
                    Q[(i, j)] = w

    bqm = dimod.BinaryQuadraticModel.from_qubo(Q)
    return bqm

## Configure AlphaQUBO API
Configure the connection details to connect to AlphaQUBO SolverAPI

In [3]:
configuration = aq.Configuration()
configuration.debug = False
configuration.host = "http://ec2-54-201-35-39.us-west-2.compute.amazonaws.com:9000"

api_instance = aq.QuboApi(aq.ApiClient(configuration))

### Read AlphaQUBO Formatted File


In [4]:
bqm = read_alphaqubo_bqm('../data/bqp2500_1.txt')
print("Size of BQM: ", len(bqm))

Size of BQM:  2500


### AlphaQUBO dimod Sampler

In [5]:
from dimod.core.sampler import Sampler
from dimod.sampleset import SampleSet

class AlphaQuboSampler(Sampler):

    properties = None
    parameters = None

    def __init__(self):
        self.properties = {}
        self.parameters = {'time_limit': [],
                           'accuracy_min': [],
                           'greediness': [],
                           'maximize': [] }

    def sample(self, bqm, api_instance, time_limit=60, accuracy_min=5, greediness=0.0, maximize=False):
        n = len(bqm.variables)
        if n == 0:
            return SampleSet.from_samples([], bqm.vartype, energy=[])

        body = aq.SolverAPI()
        body.num_vars = n
        if maximize:
            body.min_max = 1
        else:
            body.min_max = 0
        body.timeout = time_limit
        body.inputs = []
        variables = sorted(bqm.iter_variables())      
        for idx, u in enumerate(variables):
            for v in variables[idx:]:
                if u == v and bqm.linear[u]:
                    body.inputs.append(aq.Points(x=u + 1, y=u + 1, z=bqm.linear[u]))
        for idx, u in enumerate(variables):
            for v in variables[idx:]:
                if u in bqm.adj[v]:                
                    body.inputs.append(aq.Points(x=u + 1, y=v + 1, z=bqm.adj[u][v] / 2))
        body.non_zero = len(body.inputs)        
        body.parameters = "-am " + str(accuracy_min)

        try:
            # Use the inputs to define a QUBO and solve it synchronously.
            api_response = api_instance.api_qubo_solve_qubo_post(body=body)           
        except ApiException as e:
            print("Exception when calling QuboApi->api_qubo_solve_qubo_post: %s\n" % e)
        
        samples = []
        energies = []
        samples.append( api_response.solved_result )
        energies.append( api_response.solved_value )
              
        response = SampleSet.from_samples(samples, bqm.vartype, energy=energies)
        return response


### Solve using AlphaQUBO

In [6]:
# %time response = solve_aq_bqm(api_instance, bqm, timeout=60, maximize=False, params='-g 0 -am 5')
%time response = AlphaQuboSampler().sample(bqm, api_instance, maximize=True, time_limit=15)
print(response)
#%time samples = QBSolv().sample(bqm, find_max=False)
#bqm.scale(-1);
#samples = neal.SimulatedAnnealingSampler().sample(bqm)
#samples = dimod.ExactSolver().sample(bqm)
#print(samples)

Wall time: 30.6 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 2499    energy num_oc.
0  1  1  1  1  1  1  1  1  1  1  0  0  0  0  0  0  0 ...    1 1515888.0       1
['BINARY', 1 rows, 1 samples, 2500 variables]
